### 基础理论

#### Can you come up out 3 sceneraies which use AI methods? 

智能音箱、客服机器人、人脸识别。

#### How do we use Github; Why do we use Jupyter and Pycharm;

- Github 用于存储、管理、共享代码、文档及其版本。
- Jupyter 用于科研学习并且可分享代码和笔记文档。
- Pycharm 用于软件工程项目开发。

#### What's the Probability Model?

概率模型是指基于统计概率学理论而提出的相关模型。

#### Can you came up with some sceneraies at which we could use Probability Model?

- 垃圾邮件识别。
- 天气预报。

#### Why do we use probability and what's the difficult points for programming based on parsing and pattern match?

- 困难点是自然语言没有统一的规范，难以判断一个语句的合理性。
- 使用统计概率模型的原因是为了判断语句合理性。

#### What's the Language Model;

语言模型也是一种概率模型，但是基于马尔科夫假设，简化了统计计算。

#### Can you came up with some sceneraies at which we could use Language Model?


- 句子纠错、单词纠错。
- 输入法候选项、搜索提示。
- 在随机生成的文本中选择最可能是规范语句的文本。

#### What's the 1-gram language model;

句子中单词出现的概率和其它单词无关。
$$
P(sentence)=P(w_1)*P(w_2)*P(w_3)*...*P(w_n)
$$

#### What's the disadvantages and advantages of 1-gram language model;

- 优点：易于计算
- 缺点：没有考虑单词出现和上下文的关系

#### What't the 2-gram models;

句子中单词出现的概率和前面或后面的一个单词有关。
$$
P(sentence)=P(w_1)*P(w_2|w_1)*P(w_3|w_2)*...*P(w_n|w_{n-1})
$$

### 编程实践

#### 设计句子生成器

第一个语法：

In [77]:
simple_grammar = """
default => sentence
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null|Adj Adj*
verb_phrase => verb noun_phrase
Article => 一个|这个
noun => 女人|篮球|桌子|小猫
verb => 看着|听着|看见
Adj => 蓝色的|好看的|小小的|年轻的
"""

第二个语法：

In [78]:
thinking_grammar = """
default => what
what => 什么 是 名词
名词 =>  机器|思考|智能|语言
why_has => 为什么有 名词
why_do => 为什么要 动词短语
动词短语 => 动词 名词
动词 => 听说|发现|产生
when => 什么时候 动词短语
where => 在哪里 动词短语
人名 => 图灵
who => 人名 是谁
how => 怎么样 动词短语
"""

使用generae函数生成句子

In [79]:
import random

def __grammar_to_kvs(grammar_str, split_token='=>'):
    lines = grammar_str.splitlines()
    kvs = {}
    for line in lines:
        if len(line)!=0:
            k=line.split(split_token)[0].strip()
            v=line.split(split_token)[1].strip()
            kvs[k]=v
#     print(kvs)
    return kvs

def __has_children(key, kvs):
    can = False
    if key in kvs.keys() and '*' not in key and '|' not in key:
        can = True
    return can

def generate_tree(grammar_str, key):
    kvs = __grammar_to_kvs(grammar_str)
    value = kvs[key]
    if '|'  in value:
        value.replace(' ','')
    results = value.split(' ')
    for i,key in enumerate(results):
#         print('key has_childs? key=',key,'has=',__has_children(key, kvs),'i=',i,'k=',key)
        if __has_children(key, kvs):
            tmp =generate_tree(grammar_str, key)
            results[i]=tmp
        else:
            if '*' in key:
                tmp = ''
                word_type = key.split('*')[0]
                to_choice = kvs[word_type].split('|')
                words_count = random.randint(0,len(to_choice)-1) 
                while words_count>0:
                    choice_idx=random.randint(0,len(to_choice)-1)
                    choice_word=to_choice[choice_idx]
                    to_choice.remove(choice_word)
                    tmp+=choice_word
                    words_count-=1
                results[i]=tmp
            elif '|' in key:
                key_randoms = key.split('|')
#                 map(lambda e:e.strip(), key_randoms)
#                 print(key_randoms)
                tmp = key_randoms[random.randint(0,len(key_randoms)-1)]
                results[i]=tmp

    return results

def tree_to_string(root):
    result = ''
    for node in root:
        if type(node)==list:
            result+=tree_to_string(node)
        else:
            result+=node
    return result

def generate(grammar_str, key='default'):
    tree = generate_tree(grammar_str, key)
    return tree_to_string(tree)

print(generate(simple_grammar))
print(generate(simple_grammar, 'noun_phrase'))
print(generate(simple_grammar, 'verb_phrase'))
print(generate(simple_grammar, 'noun'))


一个年轻的桌子看见一个好看的小小的篮球
一个年轻的好看的小小的女人
听着这个蓝色的桌子
篮球


In [80]:
print(generate(thinking_grammar,'how'))

怎么样听说机器


定义 generate_n 拓展 generate，能够生成n个句子:

In [82]:
def generate_n(n):
    results = []
    for i in range(n):
        s = generate(simple_grammar)
        results.append(s)
#         print(s)
    return results    
    
s = generate_n(10)
print(s)

['一个小猫看见一个好看的女人', '一个蓝色的篮球看见这个年轻的小小的篮球', '这个小小的蓝色的小猫看见这个好看的小猫', '这个女人看着这个年轻的蓝色的小小的女人', '一个好看的年轻的小小的桌子看见这个好看的年轻的桌子', '这个好看的小猫看着一个女人', '这个年轻的好看的蓝色的篮球听着一个蓝色的年轻的小小的桌子', '这个篮球看见这个桌子', '这个女人看着一个蓝色的好看的小小的桌子', '这个女人看见一个小小的好看的小猫']


#### 完成语言模型的训练

- 文本数据集
    + 数据集1，保险行业问询对话集： https://github.com/Computing-Intelligence/insuranceqa-corpus-zh/raw/release/corpus/pool/train.txt.gz
    + 数据集2：豆瓣评论数据集：https://github.com/Computing-Intelligence/datasource/raw/master/movie_comments.csv

目前模型选用数据集1

In [83]:
import pandas as pd
import re


dataset = []
two_gram_dataset = []

with open('train.txt', 'r') as file_in:
    with open('train_out.txt', 'w') as file_out:
        buffer = []
        buffer_size = 100
        for line in file_in:
            ss = re.findall('[\u4e00-\u9fa5]*',line)
            ss = [s for s in ss if len(s) > 0]
            file_out.write('S')
            dataset.append('S')
            for sentence in ss:
                file_out.write(sentence)
                file_out.write('E')
                items = list(sentence)
                for item in items:
                    dataset.append(item)
                dataset.append('E')
# print(dataset[-1])

def build_two_gram(dataset, two_gram_dataset):
    for i in range(len(dataset)-1):
        two_gram_dataset.append(dataset[i]+dataset[i+1])

build_two_gram(dataset, two_gram_dataset)

# print(two_gram_dataset[-1])

def condition_prob(item, item_next, dataset, two_gram_dataset):
    count_item = dataset.count(item)
    count_two = two_gram_dataset.count(item+item_next)

    if count_two == 0 and count_item != 0:
        count_two = 1
    if count_item == 0:
        p = 1/len(dataset)
    else:
        p = count_two/count_item
        
#     print(item, item_next, count_item, count_two, p)
    return p


                
def language_model_prob(sentence):
    sentence = 'S' + sentence + 'E'
    items = list(sentence)
    p_sentence = 1
    for i in range(len(items)-1):
        p_item = condition_prob(items[i], items[i+1], dataset, two_gram_dataset) 
        p_sentence = p_sentence * p_item
    return p_sentence
        
print(language_model_prob('你好呀'))

5.937887932904423e-13


#### 获得最优质的的语言

定义 generate_best 函数，该函数输入一个语法 + 语言模型，能够生成**n**个句子，并能选择一个最合理的句子


In [90]:
def generate_best(): 
    sentences = generate_n(10)
#     print(sentences)
    sentences = sorted(sentences, key=lambda s: language_model_prob(s), reverse=True)
#     print(sentences)
#     print(language_model_prob(sentences[0]), ',' ,language_model_prob( sentences[-1]))
    return sentences[0]

generate_best()

'一个好看的蓝色的小猫看见一个桌子'

#### 这个模型存在的问题？如何提升？

- 这个模型局限于有限的数据集，很可能因为样本的限制导致模型不能准确判断有新单词的句子的规范程度。
- 提升方式：
  - 扩大数据集。
  - 修改产生模型的方式，例如采用神经网络。